# Extracción de características *Bag of Words*

Primero importamos todas las librerías necesarias

In [2]:
import pandas as pd
import numpy as np
import re
import string
import spacy
import gensim

pd.options.display.max_colwidth = None


Creamos un pequeño cuerpo de textos de ejemplo *(CORPUS)*

In [3]:
corpus = ['El cielo es azul y bonito',
          'Me encanta el cielo azul, pero no el cielo plomizo',
          'Bonito cielo hacía ese día',
          'Hoy he desayunado huevos con jamón y tostadas',
          'Juan odia las tostadas y los huevos con jamón',
          'las tostadas de jamón están muy buenas']

## Limpieza del texto
Definimos una función simple de limpieza y normalización del texto y la aplicamos a nuestro corpus.

In [4]:
nlp = spacy.load("es_core_news_sm")
def normalizar_doc(doc):
    '''Función que normaliza un texto cogiendo sólo
    las palabras en minúsculas mayores de 3 caracteres'''
    # separamos en tokens
    tokens = nlp(doc)
    # filtramos stopwords
    filtered_tokens = [t.lower_ for t in tokens if
                       len(t.text)>3 and
                       not t.is_punct]
    # juntamos de nuevo en una cadena
    doc = ' '.join(filtered_tokens)
    return doc

In [5]:
#probamos la función
normalizar_doc(corpus[0])

'cielo azul bonito'

In [6]:
corpus[0]

'El cielo es azul y bonito'

In [7]:
#aplicamos a todo el corpus
norm_corpus = [normalizar_doc(doc) for doc in corpus]
norm_corpus

['cielo azul bonito',
 'encanta cielo azul pero cielo plomizo',
 'bonito cielo hacía',
 'desayunado huevos jamón tostadas',
 'juan odia tostadas huevos jamón',
 'tostadas jamón están buenas']

In [8]:
map(normalizar_doc, corpus)

In [9]:
#alternativamente
list(map(normalizar_doc, corpus))

['cielo azul bonito',
 'encanta cielo azul pero cielo plomizo',
 'bonito cielo hacía',
 'desayunado huevos jamón tostadas',
 'juan odia tostadas huevos jamón',
 'tostadas jamón están buenas']

In [10]:
for t in map(normalizar_doc, corpus):
    print(t)

cielo azul bonito
encanta cielo azul pero cielo plomizo
bonito cielo hacía
desayunado huevos jamón tostadas
juan odia tostadas huevos jamón
tostadas jamón están buenas


# Librería `scikit-learn`
Implementamos el modelo Bag-of-Word (BoW) con `scikit-learn`

Contamos la frecuencia de aparición de los términos en cada documento, usando un vocabulario común. 

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()
cv.fit(norm_corpus) #también funcionaría cv.fit(map(normalizar_doc, corpus))

CountVectorizer()

In [12]:
type(cv)

sklearn.feature_extraction.text.CountVectorizer

In [13]:
# obtenemos palabras únicas en el corpus
vocab = cv.get_feature_names_out()
vocab

array(['azul', 'bonito', 'buenas', 'cielo', 'desayunado', 'encanta',
       'están', 'hacía', 'huevos', 'jamón', 'juan', 'odia', 'pero',
       'plomizo', 'tostadas'], dtype=object)

In [14]:
len(vocab)

15

In [15]:
cv_matrix = cv.transform(norm_corpus)
cv_matrix.shape

(6, 15)

In [16]:
#matriz sparse
cv_matrix

<6x15 sparse matrix of type '<class 'numpy.int64'>'
	with 24 stored elements in Compressed Sparse Row format>

In [17]:
#sólo guarda info de las celdas no vacías
print(cv_matrix)

  (0, 0)	1
  (0, 1)	1
  (0, 3)	1
  (1, 0)	1
  (1, 3)	2
  (1, 5)	1
  (1, 12)	1
  (1, 13)	1
  (2, 1)	1
  (2, 3)	1
  (2, 7)	1
  (3, 4)	1
  (3, 8)	1
  (3, 9)	1
  (3, 14)	1
  (4, 8)	1
  (4, 9)	1
  (4, 10)	1
  (4, 11)	1
  (4, 14)	1
  (5, 2)	1
  (5, 6)	1
  (5, 9)	1
  (5, 14)	1


In [18]:
#convertimos en matriz densa
cv_matrix = cv_matrix.toarray()
cv_matrix

array([[1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 2, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0],
       [0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1],
       [0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1]], dtype=int64)

Cada término único es una característica de la matriz generada:

In [19]:
# mostramos vectores de características BoW del corpus
pd.DataFrame(cv_matrix, columns=vocab)

,azul,bonito,buenas,cielo,desayunado,encanta,están,hacía,huevos,jamón,juan,odia,pero,plomizo,tostadas
0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,2,0,1,0,0,0,0,0,0,1,1,0
2,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,1,1,0,0,0,0,1
4,0,0,0,0,0,0,0,0,1,1,1,1,0,0,1
5,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1


El modelo genera un diccionario con todas las palabras del vocabulario y asigna un índice único a cada palabra:

In [20]:
cv.vocabulary_

{'cielo': 3,
 'azul': 0,
 'bonito': 1,
 'encanta': 5,
 'pero': 12,
 'plomizo': 13,
 'hacía': 7,
 'desayunado': 4,
 'huevos': 8,
 'jamón': 9,
 'tostadas': 14,
 'juan': 10,
 'odia': 11,
 'están': 6,
 'buenas': 2}

In [21]:
#id de las palabras del vocabulario
cv.vocabulary_.get('cielo')

3

In [22]:
#si una palabra no está en el vocabulario...
cv.vocabulary_.get('lluvia')

### Aplicando el modelo a nuevos documentos
Cuando calculamos el vector BoW de un texto nuevo con el modelo no hay que volver a ajustar el vocabulario, por lo que los términos nuevos no se tendrán en cuenta:

In [23]:
nuevo_corpus = ['El Cielo amenaza lluvia', 'Pedro desayuna tostadas de jamón con tomate']
cv_matrix_nueva = cv.transform(map(normalizar_doc, nuevo_corpus))
cv_matrix_nueva

<2x15 sparse matrix of type '<class 'numpy.int64'>'
	with 3 stored elements in Compressed Sparse Row format>

In [24]:
pd.DataFrame(cv_matrix_nueva.toarray(), columns=vocab)

,azul,bonito,buenas,cielo,desayunado,encanta,están,hacía,huevos,jamón,juan,odia,pero,plomizo,tostadas
0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1


### Modelos N-grams
Considera como términos del vocabulario cada secuencia de N palabras consecutivas que aparece en el texto (*n-gramas*).  
Por ejemplo para los *bigrams* del corpus (N=2):

In [25]:
bv = CountVectorizer(ngram_range=(2,2))
bv_matrix = bv.fit_transform(norm_corpus)

In [26]:
vocab_bigram = bv.get_feature_names_out()
vocab_bigram

array(['azul bonito', 'azul pero', 'bonito cielo', 'cielo azul',
       'cielo hacía', 'cielo plomizo', 'desayunado huevos',
       'encanta cielo', 'están buenas', 'huevos jamón', 'jamón están',
       'jamón tostadas', 'juan odia', 'odia tostadas', 'pero cielo',
       'tostadas huevos', 'tostadas jamón'], dtype=object)

In [27]:
len(vocab_bigram)

17

In [28]:
bv_matrix

<6x17 sparse matrix of type '<class 'numpy.int64'>'
	with 19 stored elements in Compressed Sparse Row format>

In [29]:
bv_matrix = bv_matrix.toarray()
pd.DataFrame(bv_matrix, columns=vocab_bigram)

,azul bonito,azul pero,bonito cielo,cielo azul,cielo hacía,cielo plomizo,desayunado huevos,encanta cielo,están buenas,huevos jamón,jamón están,jamón tostadas,juan odia,odia tostadas,pero cielo,tostadas huevos,tostadas jamón
0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,0,1,0,1,0,1,0,0,0,0,0,0,1,0,0
2,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,1,0
5,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1


Se puede establecer el rango de n-grams a `(1,2)` para obtener el conjunto de unigramas y bigramas del corpus.  
Para limitar el número de términos en el vocabulario del modelo BoW se puede limitar a los términos que aparecen en un mínimo de documentos con el parámetro `min_df`

In [30]:
bv = CountVectorizer(ngram_range=(1,2), min_df=2)
bv_matrix = bv.fit_transform(norm_corpus)

bv_matrix = bv_matrix.toarray()
vocab_bigram = bv.get_feature_names_out()
pd.DataFrame(bv_matrix, columns=vocab_bigram)

,azul,bonito,cielo,cielo azul,huevos,huevos jamón,jamón,tostadas
0,1,1,1,1,0,0,0,0
1,1,0,2,1,0,0,0,0
2,0,1,1,0,0,0,0,0
3,0,0,0,0,1,1,1,1
4,0,0,0,0,1,1,1,1
5,0,0,0,0,0,0,1,1


In [31]:
bv_matrix.shape

(6, 8)

# Librería `Gensim`
Para trabajar con la librería `Gensim` es necesario transformar los documentos en una lista de tokens.

In [32]:
def word_tokenize(text):
    return [token.text for token in nlp.make_doc(text)]

Convertimos nuestros texto de ejemplo en una lista de tokens y visualizamos todo el corpus:

In [33]:
[word_tokenize(doc) for doc in norm_corpus]

[['cielo', 'azul', 'bonito'],
 ['encanta', 'cielo', 'azul', 'pero', 'cielo', 'plomizo'],
 ['bonito', 'cielo', 'hacía'],
 ['desayunado', 'huevos', 'jamón', 'tostadas'],
 ['juan', 'odia', 'tostadas', 'huevos', 'jamón'],
 ['tostadas', 'jamón', 'están', 'buenas']]

Podemos definir una función para normalizar y al mismo tiempo tokenizar el texto:

In [34]:
def normalizar_doc_tokenize(doc):
    '''Función que normaliza un texto cogiendo sólo
    las palabras en minúsculas mayores de 3 caracteres'''
    # separamos en tokens
    tokens = nlp(doc)
    # filtramos stopwords
    filtered_tokens = [t.lower_ for t in tokens if
                       len(t.text)>3 and
                       not t.is_punct]

    return filtered_tokens

In [35]:
normalizar_doc_tokenize(corpus[0])

['cielo', 'azul', 'bonito']

In [36]:
tokenized_corpus = [normalizar_doc_tokenize(doc) for doc in corpus]
tokenized_corpus

[['cielo', 'azul', 'bonito'],
 ['encanta', 'cielo', 'azul', 'pero', 'cielo', 'plomizo'],
 ['bonito', 'cielo', 'hacía'],
 ['desayunado', 'huevos', 'jamón', 'tostadas'],
 ['juan', 'odia', 'tostadas', 'huevos', 'jamón'],
 ['tostadas', 'jamón', 'están', 'buenas']]

Forma alternativa de crear el corpus tokenizado:

In [37]:
list(map(normalizar_doc_tokenize, corpus))

[['cielo', 'azul', 'bonito'],
 ['encanta', 'cielo', 'azul', 'pero', 'cielo', 'plomizo'],
 ['bonito', 'cielo', 'hacía'],
 ['desayunado', 'huevos', 'jamón', 'tostadas'],
 ['juan', 'odia', 'tostadas', 'huevos', 'jamón'],
 ['tostadas', 'jamón', 'están', 'buenas']]

## Modelo Bag of Words
Se pasará al modelo de Gensim como:

In [38]:
from gensim.corpora import Dictionary

diccionario = Dictionary(tokenized_corpus) #alternativamente Dictionary(map(normalizar_doc_tokenize, corpus))

In [39]:
diccionario

El ID de cada palabra del diccionario se obtiene con:

In [40]:
diccionario.token2id #diccionario de id de cada palabra

{'azul': 0,
 'bonito': 1,
 'cielo': 2,
 'encanta': 3,
 'pero': 4,
 'plomizo': 5,
 'hacía': 6,
 'desayunado': 7,
 'huevos': 8,
 'jamón': 9,
 'tostadas': 10,
 'juan': 11,
 'odia': 12,
 'buenas': 13,
 'están': 14}

In [41]:
len(diccionario.token2id)

15

In [42]:
diccionario.id2token #diccionario de palabras para cada ID

{}

La librería `gensim` crea la matriz BoW con otro formato. A cada palabra distinta del corpus se le asigna un ID único. Por cada documento se genera una lista de tuplas (ID, frecuencia) con la frecuencia de aparición de cada palabra:

In [43]:
diccionario.doc2bow(tokenized_corpus[0])

[(0, 1), (1, 1), (2, 1)]

In [44]:
diccionario.token2id['plomizo'] #ID de cada término

5

In [45]:
diccionario[5] #término correspondiente a una ID

'plomizo'

Creación de la matriz BoW

In [46]:
mapped_corpus = [diccionario.doc2bow(text)
                 for text in tokenized_corpus]

In [47]:
mapped_corpus

[[(0, 1), (1, 1), (2, 1)],
 [(0, 1), (2, 2), (3, 1), (4, 1), (5, 1)],
 [(1, 1), (2, 1), (6, 1)],
 [(7, 1), (8, 1), (9, 1), (10, 1)],
 [(8, 1), (9, 1), (10, 1), (11, 1), (12, 1)],
 [(9, 1), (10, 1), (13, 1), (14, 1)]]

Por ejemplo el primer documento:

In [48]:
for (i, tf) in mapped_corpus[0]:
    print(f"{diccionario[i]}: {tf}")

azul: 1
bonito: 1
cielo: 1


In [49]:
#frec. de documentos de cada token
type(diccionario.dfs)

dict

In [50]:
diccionario.dfs

{2: 3,
 0: 2,
 1: 2,
 3: 1,
 4: 1,
 5: 1,
 6: 1,
 7: 1,
 8: 2,
 9: 3,
 10: 3,
 11: 1,
 12: 1,
 14: 1,
 13: 1}

In [51]:
for i in diccionario.dfs:
    print(f"{diccionario[i]}: {diccionario.dfs[i]}")

cielo: 3
azul: 2
bonito: 2
encanta: 1
pero: 1
plomizo: 1
hacía: 1
desayunado: 1
huevos: 2
jamón: 3
tostadas: 3
juan: 1
odia: 1
están: 1
buenas: 1


In [52]:
#frec. de aparición de cada token
for i in diccionario.cfs:
    print(f"{diccionario[i]}: {diccionario.cfs[i]}")

cielo: 4
azul: 2
bonito: 2
encanta: 1
pero: 1
plomizo: 1
hacía: 1
desayunado: 1
huevos: 2
jamón: 3
tostadas: 3
juan: 1
odia: 1
están: 1
buenas: 1


In [53]:
#alternativamente su usamos un generador
mapped_corpus = [diccionario.doc2bow(d) for d in map(normalizar_doc_tokenize, corpus)]

## Aplicación de los modelos a nuevos textos
Para aplicar un modelo BoW o TF-IDF a un nuevo documento hay que utilizar los modelos ya entrenados en `gensim` sobre el corpus original
### Modelo BoW

In [54]:
tokenized_nuevo_corpus = [normalizar_doc_tokenize(doc) for doc in nuevo_corpus]

mapped_nuevo_corpus = [diccionario.doc2bow(text)
                 for text in tokenized_nuevo_corpus]

mapped_nuevo_corpus

[[(2, 1)], [(9, 1), (10, 1)]]

In [55]:
tokenized_nuevo_corpus

[['cielo', 'amenaza', 'lluvia'],
 ['pedro', 'desayuna', 'tostadas', 'jamón', 'tomate']]

In [56]:
for (i, tf) in mapped_nuevo_corpus[1]:
    print(f"{diccionario[i]}: {tf}")

jamón: 1
tostadas: 1


In [57]:
#Más pythonico con 'map'
diccionario = Dictionary(map(normalizar_doc_tokenize, corpus))
list(map(diccionario.doc2bow, map(normalizar_doc_tokenize, nuevo_corpus)))

[[(2, 1)], [(9, 1), (10, 1)]]

In [58]:
#o mejor incluso
list(map(lambda x: diccionario.doc2bow(normalizar_doc_tokenize(x)), nuevo_corpus))

[[(2, 1)], [(9, 1), (10, 1)]]